# GeoDataFrames

This session provides an introduction to GeoDataFrames that link spatial attributes to the data in a Pandas DataFrame. Along the way we'll also learn how to download data from online databases and store them in a DataFrame.

We'll start by importing some of the general Python packages. During this session we'll make use of the pathlib library. It provides an object-oriented approach to represent filesystem paths and works across operating systems.

In [3]:
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import plotly.express as px

## Downloading data from a REST API

### SA Waterconnect data

In this next exercise we will pull online data into a Pandas DataFrame. We'll make use of the documentation of a library was developed by Kent Inverarity for obtaining groundwater data from the South Australian WaterConnect database, see
<A href="https://github.com/kinverarity1/python-sa-gwdata">https://github.com/kinverarity1/python-sa-gwdata</A>. The key command behind both libraries is the `get` method of the `requests` package, which will attempt to retrieve data from a specified source. The request is made by passing an url with a specifc stucture, which is defined by the application programming interface (API) of the service that is being queried. Several protocols exists, with the most common one today being the representational state transfer architectural style (REST), and an API that conforms to this standard is called a RESTful API.

Without providing any more technical details, let's just try to see how this works. Click the following link and observe the information that appears in your web browser.

<A href="https://www.waterconnect.sa.gov.au/_layouts/15/dfw.sharepoint.wdd/WDDDMS.ashx/GetObswellNumberSearchData?OBSNUMBER=WLG051">https://www.waterconnect.sa.gov.au/_layouts/15/dfw.sharepoint.wdd/WDDDMS.ashx/GetObswellNumberSearchData?OBSNUMBER=WLG051</A>

The data obtained appear in the form of a table and it can be seen that there are several fields. This is not very useful yet, but it gives you an idea of the information that is sent when you use the `get` method. If you look at the above url, it is possible to recognise three parts:
 * a base url: https://www.waterconnect.sa.gov.au/_layouts/15/dfw.sharepoint.wdd/WDDDMS.ashx/
 * a command: 'GetObswellNumberSearchData'
 * a section with parameters for the command: 'OBSNUMBER=WLG051'

From this we can infer that this url requests to search data based on the Obswel number, which in this case is specified to be WLG051 (not entirely coincidentally, this is the monitoring bore right next to the farm dam).

Now let's do this in Python using the `get` method. We create a variable `url`, which combines the base url and the command and we pass the command parameters as a dictionary.

In [4]:
base_url = "https://www.waterconnect.sa.gov.au/_layouts/15/dfw.sharepoint.wdd/WDDDMS.ashx/"
url = base_url + "GetObswellNumberSearchData"
rest_params = {"OBSNUMBER": "WLG051"}

response = requests.get(url, params=rest_params)

The returned data are stored in `response`. This is an object that contains information about the request and, if the request was successful, the data in JSON format (note that this differs depending on the API, other services may use a different format, e.g. csv). JSON is shorthand for JavaScript Object Notation and is a common data-interchange format. Although it is intended to be readable for humans, it is not as convenient as a DataFrame, so the next code cell converts the data to a DataFrame called `df`. 

In [ ]:
data = response.json()
df = pd.json_normalize(data)
df

Inspecting the DataFrame contents shows that it has a column 'DHNO', which stands for drillhole number. We can use this number in combination with the API command 'GetWaterLevelDetails' to get the water level time series for this well.

In [4]:
dhno = df["DHNO"]
url = base_url + "GetWaterLevelDetails"
rest_params = {"DHNO": dhno}

response = requests.get(url, params=rest_params)

Once again, we can convert the JSON data to a DataFrame. The data in the column 'OBS_DATE' can be converted to a datetime format, and be used as the index of the DataFrame.

In [ ]:
data = response.json()
df = pd.json_normalize(data)
df.index = pd.to_datetime(df["OBS_DATE"])
df

Plotting the data is then a breeze of course...

In [ ]:
fig, ax = plt.subplots()
ax.plot(df.index, df["RSWL"], '.-')
ax.set_title("WLG051")
ax.set_ylabel("Year")
ax.set_ylabel("RSWL");

The examples above demonstrate the use of the `get` method. Libraries like <A href="https://github.com/kinverarity1/python-sa-gwdata">python-sa-gwdata</A> and <A href="https://github.com/ArtesiaWater/hydropandas">Hydropandas</A> wrap Python code around this method to provide a user-friendly way to obtain data from a database. More RESTful APIs exist, and their number is growing.

### PyQGIS 
QGIS (and ArcGIS) offers Python support via the QGIS Python console. Extensive documentation is provided <A href="https://docs.qgis.org/3.28/en/docs/pyqgis_developer_cookbook/index.html#">here</A>. The example below will use GeoPandas to create a shapefile of some data downloaded from  WaterConnect. The shapefile will be imported into QGIS and PyQGIS will be used to create three separate layers showing the boreholes with chemistry, water (level) and salinity data, respectively. The PyQGIS interface will also be used to set a different marker symbol colour for each of the three layers.

Let's start by using the WaterConnect API to download the availabe boreholes in a rectangular area near McLarenVale, SA.

In [7]:
url = base_url + "GetGridData"
rest_params = {"Box": "-35.25,138.55,-35.20,138.6"}
response = requests.get(url, params=rest_params)

# Convert to DataFrame
data = response.json()
df = pd.json_normalize(data)

With Geopandas, the DataFrame can be converted to a shapefile with just a few lines of code. First the data in the columns 'LON' and 'LAT' will be used to create the coordinate data. Together with the DataFrame `df` the coordinate data in `lat_long_coordinates` is used to create a GeoDataFrame (note that the EPSG code 4326 is for lat/long coordinates based on the World Geodetic System 1984 ensemble (WSG84)). The method `to_file` saves the shapefile to disk, and the `mkdir` call ensures that the subdirectory 'shp' exists.

In [ ]:
lat_long_coordinates = gpd.points_from_xy(df["LON"], df["LAT"])
gdf = gpd.GeoDataFrame(
    df, 
    geometry=lat_long_coordinates, 
    crs="EPSG:4326",
)

Path('shp').mkdir(exist_ok=True)
gdf.to_file("shp/borehole_data.shp")

If running the code in the above cell results in an error message, you can solve it by installing the `pyogrio` by entering the following command lines in the Anaconda Prompt (note that you have to open it from the Windows Start menu, the first line below activates the geopandas_env environment):

`conda activate geopandas_env`

`conda install -c conda-forge pyogrio`

The two code cells below will only work inside the QGIS Python editor (as will be demonstrated during the session). Note that the information on the object model for QGIS is extensive. For example, the documentation for a map layer object can be found <A href="https://api.qgis.org/api/classQgsMapLayer.html">here</A> and there are many, many more (see <A href="https://api.qgis.org/api/modules.html">https://api.qgis.org/api/modules.html</A>).

In [ ]:
current_project = QgsProject.instance()

layer = current_project.mapLayersByName('borehole_data')[0]

field_names = ["CHEM", "WATER", "SAL"]

for field_name in field_names:
    new_layer = layer.clone()
    new_layer.setName(f'{field_name}_data')
    new_layer.setSubsetString(f'"{field_name}" = \'Y\'')
    current_project.addMapLayer(new_layer)

In [ ]:
# Add these lines to change the symbol colours
colors = ["red", "green", "blue"] 
for field_name, color in zip(field_names, colors):
    layer = current_project.mapLayersByName(f'{field_name}_data')[0]
    layerRenderer= layer.renderer()

    new_sym = QgsMarkerSymbol.createSimple({"color": color})
    layerRenderer.setSymbol(new_sym)
    iface.layerTreeView().refreshLayerSymbology(layer.id())

### More GeoPandas: Ovens catchment lithology

For this exercise we will use the API services of the [Geoscience Australia Digital Atlas](https://digital.atlas.gov.au/apps/cdc5e405889e419fb2939268e6395a86/explore). We will use it to download the dataset with the river regions. Like before, we pass a base url followed by some parameters. We ask for all output fields by providing an asterisk for the "outFields" parameter. The format of the returned data will be GeoJSON, as per the "f" parameter. The "where" parameter is compulsory and allows for SQL style queries (see 
https://developers.arcgis.com/rest/services-reference/enterprise/query-feature-service-layer/ for the full technical details.) In this case it is set to "1=1", which is a conditional statement that evaluates to True, so all available records are returned.

In [9]:
url = "https://services-ap1.arcgis.com/ypkPEy1AmwPKGNNv/arcgis/rest/services/River_Regions_Source_view/FeatureServer/0/query"

rest_params = {
    "outFields": "*",
    "where": "1=1",
    "f": "geojson",
}

response = requests.get(url, params=rest_params)

A GeoDataFrame can be created from the GeoJSON object by using the `from_features` method. Since the coordinate system used by the data provider is WGS 84, we set the `crs` argument to 4326, which is the EPSG code for this coordinate system. By saving the polygons to a shapefile we can inspect the downloaded data in a GIS.

In [ ]:
# Get the data part from the response
data = response.json()
# Convert the features to a GeoDataFrame
gdf_ri = gpd.GeoDataFrame.from_features(data["features"], crs=4326)
# Save to shapefile
gdf_ri.to_file("shp/river_regions.shp")

Now that we inspected the data, we can use the "where" parameter to select only the catchments within the Murray-Darling Basin.

In [5]:
url = "https://services-ap1.arcgis.com/ypkPEy1AmwPKGNNv/arcgis/rest/services/River_Regions_Source_view/FeatureServer/0/query"

rest_params = {
    "outFields": "*",
    "where": "division='Murray-Darling Basin'",
    "f": "geojson",
}

response = requests.get(url, params=rest_params)

Once again, we convert the GeoJSON data to a GeoDataFrame and save it as a shapefile.

In [ ]:
# Get the data part from the response
data = response.json()
# Convert the features to a GeoDataFrame
gdf_ri = gpd.GeoDataFrame.from_features(data["features"], crs=4326)
# Save to shapefile
gdf_ri.to_file("shp/river_regions_mdb.shp")

No let's do some analysis for a particular catchment. Let's try to calculate the percentages of the land surface covered by a certain lithology. First we select the row for the Ovens river, the catchment that we'll analyse.

In [ ]:
idx = gdf_ri["rivregname"] == "OVENS RIVER"
gdf_ov = gdf_ri.loc[idx]
gdf_ov

Next we will download the lithology data 1:1 million scale: https://digital.atlas.gov.au/search?q=lithostratigraphy&source=Geoscience%2520Australia. This cannot be done for all of Australia because the query will return too many records (in fact, the maximum number of records is limited to 1000, so it will return only a small fraction of all features in the database). Therefore will will constrain our query to a rectangular region for which we have to pass the lat/lon coordinates. We could look these up manually, but it's a lot more elegant and convenient to use the GeoDataFrame's functionality to give us these. If we type the following line, we see the minimum and maximum coordinates of a rectangular box that bounds the polygon for the Ovens catchment.

In [ ]:
gdf_ov.bounds

The above DataFrame can be converted to a text string that can be passed as part of the API call by including the "geometry" parameter. You could of course copy and paste the values above to create the string but if you were to select another catchment, you'd have to repeat this, or worse, you forget to update the string and you end up with the wrong data. So we'll make use of the `join` method of a `str`, which takes a list as an argument and stitches together the list items into a new string in which the items are separated by the original string, in this case a comma.

In [ ]:
bounds_lst = gdf_ov.bounds.values.astype(str)
bounds_lst = bounds_lst[0]
bounds_str = ",".join(bounds_lst)
bounds_str

The url and parameters are provided below. Figuring out how an API works can be quite time-consuming but for this particular case there is help available via this page https://digital.atlas.gov.au/datasets/22141b185a2942e88e9c02c93928cf9d_10/api. The code below was created using this resource and by trial and error.

In [10]:
# url = "https://services.ga.gov.au/gis/rest/services/GA_Surface_Geology/MapServer/4/query" # Scale 1:2.5 million
url = "https://services.ga.gov.au/gis/rest/services/GA_Surface_Geology/MapServer/11/query" # Scale 1:1 million

rest_params = {
    "outFields": "*",
    "where": "1=1",
    "f": "geojson",
    "geometry": bounds_str, #"116.050,-38.690,153.755,-25.726",
    "geometryType": "esriGeometryEnvelope",
    "inSR": "4326",
    "spatialRel": "esriSpatialRelIntersects",
    "outSR": "4326",
}

response = requests.get(url, params=rest_params)

And yet once again, we convert the GeoJSON data to a GeoDataFrame and save it as a shapefile.

In [ ]:
data = response.json()
gdf_li = gpd.GeoDataFrame.from_features(data["features"], crs=4326)
gdf_li.to_file("shp/lithology.shp")

The API call returned all the polygons that intersect the rectangular area that we provided and some extend outside the Ovens catchment. Luckily we can use the GeoDataFrame's `clip` method to cut away all the polygon parts that lie outside the catchment boundary.

In [ ]:
gdf_li_ov = gdf_li.clip(gdf_ov)
gdf_li_ov.to_file("shp/lithology_ovens.shp")

Now let's get the surface area for each lithology type (see https://asud.ga.gov.au/ for a descrition of the units). For this purpose the coordinate system needs to be converted to a projected coordinate system. We use the GeoDataFrame's `to_crs` method for this purpose and use the EPSG code 32755 for WGS 84 / UTM zone 55S.

In [16]:
gdf_li_ov = gdf_li_ov.to_crs(32755)

We can loop through the unique lithology unit names and select the rows from `gdf_li_ov` with that name. By specifying that we want the "geometry" column the return value of `loc` is a GeoSeries, i.e. like a Pandas Series but then containing only geometries (in this case Polygons). This GeoSeries has an attribute called `area`, which contains the area of each polygon. So by summing we obtain the total area covered by a particular lithology.

Note that the `gdf_li_ov` GeoDataFrame also has a column called "st_area(shape)" which contains the area, so we could have used that for this purpose as well. But since the goal is to showcase GeoPandas' functionalities, we will refrain from taking this shortcut.

In [19]:
litho_names = gdf_li_ov["name"].unique()

areas = []
for litho_name in litho_names:
    idx = gdf_li_ov["name"] == litho_name
    gs = gdf_li_ov.loc[idx, "geometry"]
    areas.append(gs.area.sum())

If we create a Pandas DataFrame of the lithology names and areas we can use plotly to display the result in a pie chart. In order not to jumble up the chart we rename all lithology classes with a surface area less than 100 km $^2$ to "Other". Plotly will aggregate those areas automatically and add "Other" as a category to the chart.

In [20]:
df = pd.DataFrame(
    data={
        "litho_name": litho_names,
        "area": areas,
    }
)

idx = df["area"] < 1e8
df.loc[idx, "litho_name"] = "Other"

Use the plotly.express `pie` function to create a pie chart.

In [ ]:
fig = px.pie(
    df, 
    values='area', 
    names='litho_name', 
    title='Ovens catchment lithology',
)

# Uncomment the following line to also include the lithology class name in the pie chart
# fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

## Plotting with geopandas

Geopandas makes working with geographic data a lot easier. Making plots is also part of Geopandas, based on the Matplotlib library. This makes it a breeze to make (advanced) plots for geographic data. We'll look at some simple examples here. Let's plot the geodataframe, and coloring it by the size of each sub-catchment. We can add a north arrow and a scale bar. To add a scale bar, find the Geopandas documentation here: https://geopandas.org/en/stable/gallery/matplotlib_scalebar.html


In [ ]:
fig, ax = plt.subplots()

gdf_ri.plot(column="Shape__Area", legend=True, ax=ax, edgecolor="black")

# Add the Ovens River catchment to the plot in red
gdf_ov.plot(ax=ax, color="red", alpha=0.7)

ax.set_title("MDB catchments")

# Add a North arrow
x, y, arrow_length = 0.1, 0.95, 0.15
ax.annotate(
    "N",
    xy=(x, y),
    xytext=(x, y - arrow_length),
    arrowprops=dict(facecolor="black", width=5, headwidth=15),
    ha="center",
    va="center",
    fontsize=20,
    xycoords=ax.transAxes,
)

fig.savefig("ovens_river_catchment.png")

### Interactive plotting with Geopandas

Geopandas has a nice function to explore the data, called `.explore`. This generates an interactive map of the data, such that we can more easily explore it. It sometimes takes a while to load it, but the wait is worth it.

In [ ]:
gdf_ri.explore()